In [39]:
import json

import openai
from openai.types.chat import ChatCompletionMessage

client = openai.OpenAI()
messages  = []

In [32]:
def get_weather(city: str) -> str:
    # Dummy implementation for example purposes
    return f"The current weather in {city} is sunny with a temperature of 25°C."

FUNCTION_MAP = {"get_weather": get_weather}

In [33]:
TOOLS = [
    {
        "type": "function",
        "function": {
            "name": "get_weather",
            "description": "Get the current weather in a given location",
            "parameters": {
                "type": "object",
                "properties": {
                    "city": {
                        "type": "string",
                        "description": "The name of the city to get the weather for",
                    },
                },
                "required": ["city"],
            },
        }
    }
]

In [42]:
from openai.types.chat import ChatCompletionMessage

def process_ai_response(message: ChatCompletionMessage):
    if message.tool_calls:
        messages.append({"role": "assistant",
             "content": message.content  or  "",
             "tool_calls": [
                 {
                     "id": tool_call.id,
                     "type": "function",
                     "function": {
                        "name": tool_call.function.name,
                        "arguments": tool_call.function.arguments,
                     }
                 }
                 for tool_call in message.tool_calls
             ]
        })

        for tool_call in message.tool_calls:
            function_name = tool_call.function.name
            arguments = tool_call.function.arguments

            print(f"Calling function: {function_name} with arguments: {arguments}")

            try:
                arguments = json.loads(arguments)
            except json.decoder.JSONDecodeError:
                arguments = {}

            function_to_run = FUNCTION_MAP.get(function_name)

            result = function_to_run(**arguments)

            print(f"Run {function_name} with arguments {arguments} got result: {result}")

            messages.append(
                {
                    "role": "tool",
                    "tool_call_id": tool_call.id,
                    "name": function_name,
                    "content": result,
                })

        call_ai()
    else:
        messages.append({"role": "assistant", "content": message.content})
        print(f"AI: {message.content}")

In [38]:
def call_ai():
    response = client.chat.completions.create(
            model="gpt-5-nano-2025-08-07",
            reasoning_effort="low",
            messages=messages,
            tools=TOOLS,
        )
    process_ai_response(response.choices[0].message)

In [43]:
while True:
    message = input("User: ")
    if message == 'quit' or message == 'q':
        break
    else:
        messages.append({"role": "user", "content": message})
        print(f"User: {message}")
        call_ai()

User: 내 이름은 근이야
AI: 반갑습니다, 근이님! 이 대화에서는 근이님이라고 불러드리겠습니다. 다른 호칭이 필요하시면 말씀해 주세요. 무엇을 도와드릴까요?
User: 서울 날씨는 어떄?
Calling function: get_weather with arguments: {"city":"Seoul"}
Run get_weather with arguments {'city': 'Seoul'} got result: The current weather in Seoul is sunny with a temperature of 25°C.
AI: 서울의 현재 날씨는 맑고 기온은 25°C입니다. 외출하기에 적당한 날씨예요. 필요하시면 체감온도나 바람 정보도 같이 알려드릴게요. 다른 정보가 필요하신가요?
